In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import pandas_profiling as ppf
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('max_colwidth',None)

In [2]:
#to set a drop_list to drop the columns in this list
drop_list=['glucose_min','glucose_max','wbc_min','wbc_max','platelet_max','platelet_min',
          'potassium_min','potassium_max','sodium_min','sodium_max','creatinine_min','creatinine_max',
          'bun_min','bun_max','hospital_expire_flag','has_chartevents_data',
          'expire_flag','admission_type','admission_location','discharge_location',
          'insurance','language','religion','marital_status','diagnosis','dbsource',
          'last_careunit','vasopressin_duration','sirs','apsiii','rrt','urineoutput',
          'lods','aniongap_min','aniongap_max','bands_min','bands_max', 'bicarbonate_min', 'bicarbonate_max', 'bilirubin_min',
       'bilirubin_max', 'chloride_min', 'chloride_max', 'hematocrit_min',
       'hematocrit_max','ptt_min', 'ptt_max', 'inr_min', 'inr_max', 'pt_min',
       'pt_max','spo2_mean', 'mingcs', 'infection',
       'explicit_sepsis', 'organ_dysfunction', 'mech_vent', 'angus',
       'aado2_mean', 'baseexcess_mean', 'bicarbonate_mean', 'totalco2_mean',
       'carboxyhemoglobin_mean', 'chloride_mean', 'calcium_mean', 'peep_mean',
          'methemoglobin_mean', 'o2flow_mean', 'fio2_mean',
       'so2_mean','hematocrit_mean', 'intubated_mean','requiredo2_mean', 'tidalvolume_mean',
          'sodium_mean','C-reactive protein.csv','lymphocyte_count.csv','glucose_mean',
          'lactate_min','lactate_max','temperature_mean']

In [3]:
final_tableV8=pd.read_csv('result/final_tableV5.csv')
final_tableV8.drop(drop_list, axis=1, inplace=True)
#remove children samples
#final_tableV8.drop(final_tableV8[final_tableV8['age']<18].index, inplace=True)#drop samples whose age is lesser than 18

KeyError: "['glucose_min' 'glucose_max' 'wbc_min' 'wbc_max' 'platelet_max'\n 'platelet_min' 'potassium_min' 'potassium_max' 'sodium_min' 'sodium_max'\n 'creatinine_min' 'creatinine_max' 'bun_min' 'bun_max'\n 'hospital_expire_flag' 'has_chartevents_data' 'expire_flag'\n 'admission_type' 'admission_location' 'discharge_location' 'insurance'\n 'language' 'religion' 'marital_status' 'diagnosis' 'dbsource'\n 'last_careunit' 'vasopressin_duration' 'aniongap_min' 'aniongap_max'\n 'bands_min' 'bands_max' 'bicarbonate_min' 'bicarbonate_max'\n 'bilirubin_min' 'bilirubin_max' 'chloride_min' 'chloride_max'\n 'hematocrit_min' 'hematocrit_max' 'ptt_min' 'ptt_max' 'inr_min' 'inr_max'\n 'pt_min' 'pt_max' 'spo2_mean' 'aado2_mean' 'baseexcess_mean'\n 'bicarbonate_mean' 'totalco2_mean' 'carboxyhemoglobin_mean'\n 'chloride_mean' 'calcium_mean' 'peep_mean' 'methemoglobin_mean'\n 'o2flow_mean' 'fio2_mean' 'so2_mean' 'hematocrit_mean' 'intubated_mean'\n 'requiredo2_mean' 'tidalvolume_mean' 'sodium_mean'\n 'C

In [ ]:
final_tableV8['hemoglobin_mean']=(final_tableV8['hemoglobin_min']+final_tableV8['hemoglobin_min'])/2
final_tableV8['albumin']=(final_tableV8['albumin_max']+final_tableV8['albumin_min'])/2
final_tableV8['MAP']=((1/3)*final_tableV8['sysbp_mean'] + (2/3)*final_tableV8['diasbp_mean'])
final_tableV8.drop(['hemoglobin_min','hemoglobin_max','albumin_max','albumin_min',
                   'sysbp_mean','diasbp_mean'], axis=1, inplace=True)
#missing_values=list(final_tableV8.columns[20:22])+list(final_tableV8.columns[37:45])+list(final_tableV8.columns[51:])

In [ ]:
#use winsorize to deal with outlier
def winsorize(data, percentage_one=0.01, percentage_two=0.99, show=False):
    if show==True:
        plt.figure(figsize=(5,5), dpi=300)
        ax1=plt.subplot(121)
        plt.hist(data, bins=100)
    count1=len(data)
    lower_bound=data.quantile(percentage_one)
    upper_bound=data.quantile(percentage_two)
    count2=len(data[data<lower_bound])+len(data[data>upper_bound])
    data[data<lower_bound]=lower_bound
    data[data>upper_bound]=upper_bound
    
    if show==True:
        print('', data.name, ':\n', 'lower bound is', lower_bound,'\n',
          'upeer bound is', upper_bound,'\n', 
          'totally dealt with',count2, 'of', count1, '(', (count2/count1)*100, '%)', '\n')
        ax2=plt.subplot(122)
        plt.hist(data, bins=100)
    
    
def regression(name1,name2):
    name_x=name1
    name_y=name2
    regData=final_tableV8[final_tableV8[name_x].notnull() & final_tableV8[name_y].notnull()]
    x=regData[name_x].values.reshape(-1,1)
    y=regData[name_y].values.reshape(-1,1)
    reg=LinearRegression()
    reg.fit(x,y)
    x2=np.linspace(x.min(), x.max(), num=10)
    y2 = reg.predict(x2.reshape(-1,1))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,1,1)
    ax1.scatter(x,y, s=2)
    ax1.plot(x2, y2, c='black')
    plt.show()
    print('R^2 is', reg.score(x,y))

In [ ]:
winsorize(final_tableV8['age'], 0.00, 0.948)
winsorize(final_tableV8['height_first'], 0.01, 0.99)
winsorize(final_tableV8['weight_first'], 0.01, 0.99)

In [ ]:
def meanFill(dataframe, variables_list, show=False):
    for i in variables_list:
        dataframe[i].fillna(dataframe[i].mean(),inplace=True)


def fill(dataframe, variables_list, variables_y, parameters, model, show=False):
    for i in variables_y:
        if show==True:
            plt.figure(figsize=(10,10), dpi=300)
            ax1=plt.subplot(121)
            plt.hist(dataframe[i], bins=100)
        clf=GridSearchCV(model, parameters, cv=5)
        X=dataframe[dataframe[i].notnull()][variables_list]
        y=dataframe[dataframe[i].notnull()][i]
        clf.fit(X, y)
        scores=clf.score(X, y)
        #scores = cross_val_score(estimator=model, X=dataframe[variables_list], y=dataframe['hospital_mortality'], cv=5,scoring = "neg_mean_absolute_error")
        #train_score=model.score(dataframe[variables_list], dataframe['hospital_mortality'])

        print('the best model is:', clf.best_estimator_, 'with best score', clf.best_score_)
        print('mean test scores are', clf.cv_results_['mean_test_score'])
        fillX=dataframe[dataframe[i].isnull()][variables_list]
        dataframe.loc[dataframe[dataframe[i].isnull()].index, i]=clf.predict(fillX)
        if show==True:
            ax2=plt.subplot(122)
            plt.hist(dataframe[i], bins=100)
            plt.show()

In [ ]:
final_tableV8.describe()

In [ ]:
final_tableV8['los'].fillna(final_tableV8['los'].mean(), inplace=True)
#final_tableV8['height_first'].fillna(final_tableV8['height_first'].mean(), inplace=True)
final_tableV8['weight_first'].fillna(final_tableV8['weight_first'].mean(), inplace=True)
#variables_list=final_tableV8.columns[29:]
#meanFill(final_tableV8, missing_values)
final_tableV8['gender']=final_tableV8['gender'].map(lambda x:1 if x=='M' else 0)
divide=final_tableV8['height_first']/100
final_tableV8['BMI']=final_tableV8['weight_first']/(divide*divide)
final_tableV8.drop(['weight_first','height_first'], axis=1, inplace=True)

In [ ]:
data=final_tableV8[final_tableV8['sepsis.csv']==1]
data=data[data['uric_acid.csv'].notnull()]

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
missing_values=['los','hospital_mortality', '30_mortality',
       '90_mortality', '365_mortality', 'age', 'sofa', 'vasopressin', 'sapsii',
       'heartrate_mean', 'resprate_mean', 'tempc_mean', 'hemoglobin_mean',
       'lactate_mean', 'pco2_mean', 'ph_mean', 'po2_mean', 'potassium_mean',
       'CKD.csv', 'COPD.csv', 'coronary heart disease.csv', 'diabetes.csv',
       'hypertension.csv', 'liver disease.csv', 'sepsis.csv', 'creatinine.csv',
       'glucose.csv', 'neutrophils.csv', 'platelet.csv', 'potassium.csv',
       'sodium.csv', 'urea nitrogen.csv', 'uric_acid.csv', 'WBC.csv',
       'albumin', 'MAP', 'BMI']
meanFill(data, missing_values)

In [ ]:
data.to_csv('result/final_tableV10.csv', index=False)